<a href="https://colab.research.google.com/github/suyash2104/Disaster-detector/blob/master/Bert_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from transformers import BertForSequenceClassification, AdamW
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model_save_name = 'my_model.pt'
path = F"/content/drive/My Drive/disaster tweets/{model_save_name}"
model.load_state_dict(torch.load(path))
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
encoded = torch.load("/content/drive/My Drive/disaster tweets/encoded.pt")

In [ ]:
Xtest_id = torch.load("/content/drive/My Drive/disaster tweets/Xtest_id.pt")
Xtest_am = torch.load("/content/drive/My Drive/disaster tweets/Xtest_am.pt")

In [ ]:
df1 = pd.read_csv('/content/drive/My Drive/disaster tweets/train.csv')

In [ ]:
labels = df1.target.iloc[6800:7613]
labels = torch.tensor(labels.values.tolist())
labels = labels.type(torch.LongTensor)
labels.shape

torch.Size([813])

In [ ]:
def calc_accuracy(mdl,X,Y):
    """Calculates model accuracy
    
    Arguments:
        mdl {nn.model} -- nn model
        X {torch.Tensor} -- input data
        Y {torch.Tensor} -- labels/target values
    
    Returns:
        [torch.Tensor] -- accuracy
    """
    max_vals, max_indices = torch.max(mdl(X),1)
    n = max_indices.size(0) #index 0 for extracting the # of elements
    train_acc = (max_indices == Y).sum(dtype=torch.float32)/n
    return train_acc

In [ ]:
model.eval()     #model in testing mode
batch_size = 32
permutation = torch.randperm(Xtest_id.size()[0])

for i in range(0,Xtest_id.size()[0], batch_size):

  indices = permutation[i:i+batch_size]
  batch_x, batch_y, batch_am = Xtest_id[indices], labels[indices], Xtest_am[indices]

        # in case you wanted a semi-full example
  outputs = model(batch_x, attention_mask=batch_am, labels=batch_y)
  loss = outputs[0]
  print('Loss:' ,loss)

Loss: tensor(0.2756, grad_fn=<NllLossBackward>)
Loss: tensor(0.3205, grad_fn=<NllLossBackward>)
Loss: tensor(0.1968, grad_fn=<NllLossBackward>)
Loss: tensor(0.4183, grad_fn=<NllLossBackward>)
Loss: tensor(0.5287, grad_fn=<NllLossBackward>)
Loss: tensor(0.4672, grad_fn=<NllLossBackward>)
Loss: tensor(0.1713, grad_fn=<NllLossBackward>)
Loss: tensor(0.3473, grad_fn=<NllLossBackward>)
Loss: tensor(0.3690, grad_fn=<NllLossBackward>)
Loss: tensor(0.3671, grad_fn=<NllLossBackward>)
Loss: tensor(0.4025, grad_fn=<NllLossBackward>)
Loss: tensor(0.2439, grad_fn=<NllLossBackward>)
Loss: tensor(0.5805, grad_fn=<NllLossBackward>)
Loss: tensor(0.7118, grad_fn=<NllLossBackward>)
Loss: tensor(0.3429, grad_fn=<NllLossBackward>)
Loss: tensor(0.3250, grad_fn=<NllLossBackward>)
Loss: tensor(0.4184, grad_fn=<NllLossBackward>)
Loss: tensor(0.5494, grad_fn=<NllLossBackward>)
Loss: tensor(0.2730, grad_fn=<NllLossBackward>)
Loss: tensor(0.5022, grad_fn=<NllLossBackward>)
Loss: tensor(0.4983, grad_fn=<NllLossBac

In [ ]:
outputs[1]
import torch.nn.functional as F
pt_predictions = F.softmax(outputs[1], dim=-1)
pt_predictions.tolist()

[[0.044832587242126465, 0.9551674127578735],
 [0.28356072306632996, 0.7164392471313477],
 [0.6921232342720032, 0.3078767955303192],
 [0.83025723695755, 0.16974276304244995],
 [0.053857941180467606, 0.9461420774459839],
 [0.6431791186332703, 0.3568209111690521],
 [0.9055293202400208, 0.09447063505649567],
 [0.9155734777450562, 0.08442649245262146],
 [0.0556192621588707, 0.9443807005882263],
 [0.8098210096359253, 0.19017894566059113],
 [0.09144947677850723, 0.9085505604743958],
 [0.0671040490269661, 0.9328959584236145],
 [0.9072011709213257, 0.09279881417751312]]

In [ ]:
original_test_id = encoded['input_ids'][7613:]
original_test_am = encoded['attention_mask'][7613:]

In [ ]:
original_test_id.size()

torch.Size([3263, 84])

In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.  
    
    device = torch.device('cuda')    


    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [ ]:
import torch.nn.functional as F  #for softmax function    
batch_size = 32
prediction = np.empty((0,2)) #empty numpy for appending our output
ids = torch.tensor(range(original_test_id.size()[0]))
for i in range(0,original_test_id.size()[0], batch_size):
  indices = ids[i:i+batch_size]
  batch_x1, batch_am1 = original_test_id[indices], original_test_am[indices]
  pred = model(batch_x1, batch_am1) #Here only activation is given as output
  pt_predictions = F.softmax(pred[0], dim=-1)  #applying softmax activation function
  prediction = np.append(prediction, pt_predictions.detach().numpy(), axis=0) #appending the prediction

In [ ]:
prediction.shape

(3263, 2)

In [ ]:
sub = np.argmax(prediction, axis=1)

In [ ]:
test = pd.read_csv('/content/drive/My Drive/disaster tweets/test.csv')
len(test)

3263

In [ ]:
submission = pd.DataFrame({'id': test.id.values, 'target': sub} )

In [ ]:
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
submission.to_csv('/content/drive/My Drive/disaster tweets/submission.csv', index=False)